In [1]:
"""
Solving FrozenLake8x8 environment using Value-Itertion.
Author : Moustafa Alzantot (malzantot@ucla.edu)
"""
import numpy as np
import gym
from gym import wrappers


def run_episode(env, policy, gamma = 1.0, render = False):
    """ Evaluates policy by using it to run an episode and finding its
    total reward.
    args:
    env: gym environment.
    policy: the policy to be used.
    gamma: discount factor.
    render: boolean to turn rendering on/off.
    returns:
    total reward: real value of the total reward recieved by agent under policy.
    """
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    while True:
        if render:
            env.render()
        obs, reward, done , _ = env.step(int(policy[obs]))
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
    return total_reward

In [2]:
def evaluate_policy(env, policy, gamma = 1.0,  n = 100):
    """ Evaluates a policy by running it n times.
    returns:
    average total reward
    """
    scores = [
            run_episode(env, policy, gamma = gamma, render = False)
            for _ in range(n)]
    return np.mean(scores)

In [3]:
#策略选取
def extract_policy(v, gamma = 1.0):
    """ Extract the policy given a value-function """
    policy = np.zeros(env.nS)  # 初始化存储策略的数组
    for s in range(env.nS):   # 对每个状态构建Q表，并在该状态下对每个行为计算Q值，
        q_sa = np.zeros(env.action_space.n)   # 初始化Q表
        for a in range(env.action_space.n):   # 对每个动作计算
            for next_sr in env.P[s][a]:
                # next_sr is a tuple of (probability, next state, reward, done)
                p, s_, r, _ = next_sr
                q_sa[a] += [sum(p * (r + gamma * v[s_]) for p, s_, r, _ in env.P[s][a])]#根据ppt RL2-1 4页的Bellman方程的第二个方法，计算q_sa  更新Q表，即更新动作对应的Q值（4个动作分别由0-3表示）
        policy[s] = np.argmax(q_sa)         #贪婪式的更新policy ppt RL2-1 21页
    return policy

In [4]:
#值迭代
def value_iteration(env, gamma = 1.0):
    """ Value-iteration algorithm """
    v = np.zeros(env.nS)  # initialize value-function初始化状态值表（V表）
    max_iterations = 100000  # 迭代次数
    eps = 1e-20   # 收敛判断阈值
    for i in range(max_iterations):   # 开始迭代
        prev_v = np.copy(v)      # 初始化更新后的V表（旧表复制过来）
        # 计算每个状态下所有行为的next_state_rewards,并更新状态动作值表（Q表），最后取最大Q值更新V表
        for s in range(env.nS):    # 遍历每个状态
            for a in range(env.nA):   # 初始化存储下一个状态的奖励的列表
                q_sa = [sum(p * (r + gamma * v[s_]) for p, s_, r, _ in env.P[s][a])] #根据ppt RL2-1 28页的方法，计算v
                v[s] = max(q_sa) 
        if (np.sum(np.fabs(prev_v - v)) <= eps):
            print ('Value-iteration converged at iteration# %d.' %(i+1))
            break
    return v

In [5]:
if __name__ == '__main__':
    env_name  = 'FrozenLake8x8-v0'
    gamma = 1.0
    env = gym.make(env_name)
    optimal_v = value_iteration(env, gamma = 1.0)   #使用value迭代法得到最优的v值    最优值函数
    policy = extract_policy(optimal_v, gamma = 1.0)    #根据最好的v值，获取最优policy     最优质策略
    policy_score = evaluate_policy(env, policy, gamma, n=100)
    print('Policy average score = ', policy_score)

Value-iteration converged at iteration# 15172.
Policy average score =  0.63
